In [5]:
from itertools import combinations

def get_frequent_itemsets(transactions, min_support):
    """Generates frequent itemsets using the Apriori algorithm."""
    itemsets = {}
    num_transactions = len(transactions)
    single_items = set(item for transaction in transactions.values() for item in transaction)

    # Count support for single items
    item_counts = {item: sum(1 for transaction in transactions.values() if item in transaction) for item in single_items}

    # Convert min_support to a fraction if given as an absolute count
    min_support = min_support / num_transactions if min_support > 1 else min_support

    for item, count in item_counts.items():
        support = count / num_transactions
        if support >= min_support:
            itemsets[frozenset([item])] = support

    k = 2
    current_itemsets = set(itemsets.keys())

    while current_itemsets:
        candidate_itemsets = set(
            frozenset(a | b) for a in current_itemsets for b in current_itemsets if len(a | b) == k
        )

        new_frequent_itemsets = {}
        for itemset in candidate_itemsets:
            support_count = sum(1 for transaction in transactions.values() if itemset.issubset(transaction))
            support = support_count / num_transactions
            if support >= min_support:
                new_frequent_itemsets[itemset] = support

        if not new_frequent_itemsets:
            break

        itemsets.update(new_frequent_itemsets)
        current_itemsets = set(new_frequent_itemsets.keys())
        k += 1

    return itemsets

def generate_association_rules(frequent_itemsets, min_confidence):
    """Generates association rules from frequent itemsets."""
    rules = []

    for itemset, support in frequent_itemsets.items():
        if len(itemset) < 2:
            continue

        for k in range(1, len(itemset)):
            for subset in combinations(itemset, k):
                subset = frozenset(subset)
                remaining = itemset - subset
                if remaining:
                    confidence = support / frequent_itemsets[subset]
                    if confidence >= min_confidence:
                        rules.append((subset, remaining, confidence))

    return rules

# Example Usage with TID
transactions = {
    'T100': {'1', '3', '5'},
    'T200': {'2', '3', '5'},
    'T300': {'1', '2', '3', '5'},
    'T400': {'2', '5'},
    'T500': {'1', '3', '5'}
}

min_support = 2  # Can be an absolute count or a fraction (e.g., 0.5)
min_confidence = 0.7

frequent_itemsets = get_frequent_itemsets(transactions, min_support)
rules = generate_association_rules(frequent_itemsets, min_confidence)

print("Frequent Itemsets:")
for itemset, support in frequent_itemsets.items():
    print(f"{set(itemset)}: {support:.2f}")

print("\nAssociation Rules:")
for antecedent, consequent, confidence in rules:
    print(f"{set(antecedent)} -> {set(consequent)} (Confidence: {confidence:.2f})")


Frequent Itemsets:
{'3'}: 0.80
{'2'}: 0.60
{'1'}: 0.60
{'5'}: 1.00
{'3', '1'}: 0.60
{'5', '1'}: 0.60
{'5', '2'}: 0.60
{'3', '2'}: 0.40
{'3', '5'}: 0.80
{'3', '1', '5'}: 0.60
{'3', '5', '2'}: 0.40

Association Rules:
{'3'} -> {'1'} (Confidence: 0.75)
{'1'} -> {'3'} (Confidence: 1.00)
{'1'} -> {'5'} (Confidence: 1.00)
{'2'} -> {'5'} (Confidence: 1.00)
{'3'} -> {'5'} (Confidence: 1.00)
{'5'} -> {'3'} (Confidence: 0.80)
{'3'} -> {'5', '1'} (Confidence: 0.75)
{'1'} -> {'3', '5'} (Confidence: 1.00)
{'3', '1'} -> {'5'} (Confidence: 1.00)
{'3', '5'} -> {'1'} (Confidence: 0.75)
{'5', '1'} -> {'3'} (Confidence: 1.00)
{'3', '2'} -> {'5'} (Confidence: 1.00)
